In [1]:
import os
import sys
import importlib
PARENT_DIR = os.path.realpath('..')+'/'
sys.path.append(PARENT_DIR)
  
import numpy as np
from tqdm import tqdm_notebook
from joblib import Parallel, delayed
import subprocess
from pathlib import Path

try: importlib.reload(sl)
except: import synt_lib as sl

In [2]:
DIRS = sl.get_dirs(parent_dir=PARENT_DIR)
M_PARAMS = sl.get_model_params()

# Get all mp3 files

In [3]:
def convert_file(path, ext_to='.wav'):
    if isinstance(path, Path):
        path = path.as_posix()
    ext_from = '.'+path.split('.')[-1]
    newpath = path.replace(ext_from, ext_to)
    if os.path.isfile(newpath):
        return 2, newpath
    
    try:
        subprocess.run(['ffmpeg', '-loglevel', 'panic', '-i', path, '-ar', str(M_PARAMS['SAMPLE_RATE']), newpath])
        return 1, newpath
    except Exception as e:
        print(e)
        return 0, newpath

Remove old wav files

In [4]:
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")
res_len = len(list(result))
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")

In [5]:
for fname in tqdm_notebook(result, total=res_len):
    newfname = fname.as_posix().replace('.mp3','.wav')
    if os.path.isfile(newfname):
        subprocess.run(['rm', newfname])

HBox(children=(IntProgress(value=0, max=14), HTML(value='')))

Convert to new wav files

In [6]:
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")
res_len = len(list(result))
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")

In [7]:
res_len

14

In [8]:
%%time
res_data = Parallel(n_jobs=4, verbose=4)(delayed(convert_file)(path) for path in result);

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: user 83 ms, sys: 62.6 ms, total: 146 ms
Wall time: 2.45 s


[Parallel(n_jobs=4)]: Done  11 out of  14 | elapsed:    2.2s remaining:    0.6s
[Parallel(n_jobs=4)]: Done  14 out of  14 | elapsed:    2.4s finished


In [9]:
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")
res_len = len(list(result))
result = Path(DIRS['RAW_DATA']).rglob("*.mp3")

In [10]:
result_wav = Path(DIRS['RAW_DATA']).rglob("*.wav")
res_len_wav = len(list(result_wav))
result_wav = Path(DIRS['RAW_DATA']).rglob("*.wav")

In [11]:
res_len, res_len_wav

(14, 14)

А, нет. Всё отработало.